In [1]:
import os
import pickle
import numpy as np
import xml.etree.ElementTree as ET
import random
import svgwrite
from IPython.display import SVG, display
import tensorflow as tf
import time
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from utils import *

In [2]:
class FakeArgParse():
    def __init__(self):
        pass
args = FakeArgParse()

args.train = True

args.rnn_size = 10
args.tsteps = 50
args.nmixtures = 5

args.kmixtures = 1
args.alphabet = ' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
args.tsteps_per_ascii = 25

args.batch_size = 1
args.nepochs = 250
args.dropout = 0.8
args.nbatches = 500

args.grad_clip = 10
args.learning_rate = 1e-4
args.lr_decay = 0.95

args.data_scale = 20
args.data_dir = './data'
args.log_dir = './output'
args.save_path = './output/model.ckpt'
args.save_every = 50

In [3]:
class Model():
    def __init__(self, args, bias=0, infer=False):
        self.args = args
        self.char_vec_len = len(args.alphabet) + 1
        self.train = args.train
        self.dropout = args.dropout
        self.ascii_steps = args.tsteps/args.tsteps_per_ascii
        if infer:
            args.batch_size = 1
            args.seq_length = 1

        tf.reset_default_graph()

        self.input_data = tf.placeholder(tf.float32, [None,args.tsteps,3], name='x_batch')
        self.target_data = tf.placeholder(tf.float32, [None,args.tsteps,3], name='y_batch')
        self.char_seq = tf.placeholder(tf.float32, [None, self.ascii_steps, self.char_vec_len], name='input_ascii')

        self.graves_initializer = tf.truncated_normal_initializer(mean=0., stddev=.075, seed=None, dtype=tf.float32)
        self.window_b_initializer = tf.truncated_normal_initializer(mean=-3.0, stddev=.25, seed=None, dtype=tf.float32)

        NOUT = 1 + args.nmixtures * 6

        W = tf.get_variable("W", [args.rnn_size,NOUT], initializer=self.graves_initializer)
        b = tf.get_variable("b", [NOUT], initializer=self.graves_initializer)
        W_attn = tf.get_variable("W_attn", [args.rnn_size,args.kmixtures*3], initializer=self.graves_initializer)
        b_attn = tf.get_variable("b_attn", [args.kmixtures*3], initializer=self.window_b_initializer)

        cell_func = tf.contrib.rnn.LSTMCell # could be GRUCell or RNNCell
        def lstm_cell():
            if (self.train and self.dropout < 1): # training mode
                return tf.contrib.rnn.DropoutWrapper(cell_func(args.rnn_size, state_is_tuple=True, initializer=self.graves_initializer), output_keep_prob = self.dropout)
            else:
                return cell_func(args.rnn_size, state_is_tuple=True, initializer=self.graves_initializer)

        self.cell0 = lstm_cell()
        self.istate_cell0 = self.cell0.zero_state(args.batch_size, tf.float32)

        inputs = [tf.squeeze(input_, [1]) for input_ in tf.split(self.input_data, args.tsteps, 1)]

        outs_cell0, self.fstate_cell0 = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, self.istate_cell0, self.cell0, loop_function=None, scope='cell0')
        
        def get_phi(alpha, beta, kappa):
            u = np.arange(self.ascii_steps)
            kappa_term = tf.square(tf.subtract(kappa,u))
            exp_term = tf.multiply(-beta,kappa_term)
            phi_k = tf.multiply(alpha, tf.exp(exp_term))
            phi = tf.reduce_sum(phi_k,1, keep_dims=True)
            return phi
        
        self.init_kappa = tf.placeholder(dtype=tf.float32, shape=[None, args.kmixtures, 1]) 
        prev_kappa = self.init_kappa

        for t in range(args.tsteps):
            attn = tf.matmul(outs_cell0[t], W_attn) + b_attn
            attn_k = tf.exp(tf.reshape(attn, [-1, 3*args.kmixtures,1]))
            [alpha, beta, kappa] = tf.split(attn_k, 3, axis=1)
            kappa = kappa + prev_kappa
            phi = get_phi(alpha, beta, kappa)
            c = tf.matmul(phi,self.char_seq)
            c = tf.squeeze(c, [1])
            outs_cell0[t] = tf.concat([outs_cell0[t], c, inputs[t]],1)
            prev_kappa = kappa

        self.window = c
        self.phi = phi
        #print self.output_1.shape

        self.new_kappa = prev_kappa

        
        self.cell1 = lstm_cell()
        self.istate_cell1 = self.cell1.zero_state(args.batch_size, tf.float32)
        outs_cell1, self.fstate_cell1 = tf.contrib.legacy_seq2seq.rnn_decoder(outs_cell0, self.istate_cell1, self.cell1, loop_function=None, scope='cell1')

        self.cell2 = lstm_cell()
        self.istate_cell2 = self.cell2.zero_state(args.batch_size, tf.float32)
        outs_cell2, self.fstate_cell2 = tf.contrib.legacy_seq2seq.rnn_decoder(outs_cell1, self.istate_cell2, self.cell2, loop_function=None, scope='cell2')

        states_series = tf.reshape(tf.concat(outs_cell2,1), [-1, args.rnn_size])
        #self.state_out = tf.identity(current_state, name='state_out')
        output = tf.matmul(states_series, W) + b
        #[x1, x2, eos] = tf.split(axis=1, num_or_size_splits=3, value=flat_data)
        #eos = tf.sigmoid(eos)

        flat_target_data = tf.reshape(self.target_data,[-1, 3])
        [x1_data, x2_data, eos_data] = tf.split(axis=1, num_or_size_splits=3, value=flat_target_data)

        #x1_loss = tf.losses.mean_squared_error(x1_data, x1)
        #x2_loss = tf.losses.mean_squared_error(x2_data, x2)
        #eos_loss = tf.losses.softmax_cross_entropy(eos_data, eos)
        
        def tf_2d_normal(x1, x2, mu1, mu2, s1, s2, rho):
            # eq # 24 and 25 of http://arxiv.org/abs/1308.0850
            norm1 = tf.subtract(x1, mu1)
            norm2 = tf.subtract(x2, mu2)
            s1s2 = tf.multiply(s1, s2)
            z = tf.square(tf.div(norm1, s1))+tf.square(tf.div(norm2, s2))-2*tf.div(tf.multiply(rho, tf.multiply(norm1, norm2)), s1s2)
            negRho = 1-tf.square(rho)
            result = tf.exp(tf.div(-z,2*negRho))
            denom = 2*np.pi*tf.multiply(s1s2, tf.sqrt(negRho))
            result = tf.div(result, denom)
            return result

        def get_lossfunc(z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr, z_eos, x1_data, x2_data, eos_data):
            result0 = tf_2d_normal(x1_data, x2_data, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr)
            # implementing eq # 26 of http://arxiv.org/abs/1308.0850
            epsilon = 1e-20
            result1 = tf.multiply(result0, z_pi)
            result1 = tf.reduce_sum(result1, 1, keep_dims=True)
            result1 = -tf.log(tf.maximum(result1, 1e-20)) # at the beginning, some errors are exactly zero.

            result2 = tf.multiply(z_eos, eos_data) + tf.multiply(1-z_eos, 1-eos_data)
            result2 = -tf.log(result2)

            result = result1 + result2
            return tf.reduce_sum(result)

        # below is where we need to do MDN splitting of distribution params
        def get_mixture_coef(output):
            # returns the tf slices containing mdn dist params
            # ie, eq 18 -> 23 of http://arxiv.org/abs/1308.0850
            z = output
            z_eos = z[:, 0:1]
            z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr = tf.split(axis=1, num_or_size_splits=6, value=z[:, 1:])

            # process output z's into MDN paramters

            # end of stroke signal
            z_eos = tf.sigmoid(z_eos) # should be negated, but doesn't matter.

            b=0 # BIAS

            # softmax all the pi's:
            max_pi = tf.reduce_max(z_pi, 1, keep_dims=True)
            z_pi = tf.subtract(z_pi, max_pi)
            z_pi = tf.exp(z_pi*(1+b))
            normalize_pi = tf.reciprocal(tf.reduce_sum(z_pi, 1, keep_dims=True))
            z_pi = tf.multiply(normalize_pi, z_pi)

            # exponentiate the sigmas and also make corr between -1 and 1.
            z_sigma1 = tf.exp(z_sigma1-b)
            z_sigma2 = tf.exp(z_sigma2-b)
            z_corr = tf.tanh(z_corr)

            return [z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr, z_eos]


        [o_pi, o_mu1, o_mu2, o_sigma1, o_sigma2, o_corr, o_eos] = get_mixture_coef(output)

        self.pi = o_pi
        self.mu1 = o_mu1
        self.mu2 = o_mu2
        self.sigma1 = o_sigma1
        self.sigma2 = o_sigma2
        self.corr = o_corr
        self.eos = o_eos

        lossfunc = get_lossfunc(o_pi, o_mu1, o_mu2, o_sigma1, o_sigma2, o_corr, o_eos, x1_data, x2_data, eos_data)
        self.cost = lossfunc / (args.batch_size * args.tsteps)

        self.learning_rate = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars), 10.)
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))
        
        self.sess = tf.InteractiveSession()
        self.saver = tf.train.Saver(tf.global_variables())
        self.sess.run(tf.global_variables_initializer())
        

In [4]:
def train_model(args):
    logger = Logger(args) # make logging utility
    logger.write("\nTRAINING MODE...")
    logger.write("{}\n".format(args))
    logger.write("loading data...")
    data_loader = DataLoader(args, logger=logger)

    logger.write("building model...")
    model = Model(args)

    v_x, v_y, v_s, v_c = data_loader.validation_data()
    valid_inputs = {model.input_data: v_x, model.target_data: v_y, model.char_seq: v_c}

    logger.write("training...")
    for e in range(args.nepochs):
        model.sess.run(tf.assign(model.learning_rate, args.learning_rate * (args.lr_decay ** e)))
        logger.write("learning rate: {}".format(model.learning_rate.eval()))

        c0, c1, c2 = model.istate_cell0.c.eval(), model.istate_cell1.c.eval(), model.istate_cell2.c.eval()
        h0, h1, h2 = model.istate_cell0.h.eval(), model.istate_cell1.h.eval(), model.istate_cell2.h.eval()
        kappa = np.zeros((args.batch_size, args.kmixtures, 1))

        for b in range(args.nbatches):

            i = e * args.nbatches + b

            if i % args.save_every == 0 and (i > 0):
                model.saver.save(model.sess, args.save_path) ; logger.write('SAVED MODEL')
                with open(args.log_dir + '/checkpoint', "w") as raw:
                    raw.write('model_checkpoint_path: "model"\nall_model_checkpoint_paths: "model"')


            start = time.time()
            x, y, s, c = data_loader.next_batch()

            feed = {model.input_data: x, model.target_data: y, model.char_seq: c, model.init_kappa: kappa, \
              model.istate_cell0.c: c0, model.istate_cell1.c: c1, model.istate_cell2.c: c2, \
              model.istate_cell0.h: h0, model.istate_cell1.h: h1, model.istate_cell2.h: h2}

            [train_loss, _] = model.sess.run([model.cost, model.train_op], feed)

            end = time.time()
            if i % 10 is 0: logger.write("{}/{}, loss = {:.3f}, time = {:.3f}" \
                .format(i, args.nepochs * args.nbatches, train_loss, end - start) )


In [5]:
train_model(args)


TRAINING MODE...
<__main__.FakeArgParse instance at 0x7f9ef50d26c8>

loading data...
	loaded dataset:
		11321 train individual data points
		595 valid individual data points
		11321 batches
building model...
training...
learning rate: 9.99999974738e-05
0/125000, loss = 2.736, time = 2.361
10/125000, loss = 3.552, time = 0.017
20/125000, loss = 5.789, time = 0.016
30/125000, loss = 3.133, time = 0.017
40/125000, loss = 4.344, time = 0.016
SAVED MODEL
50/125000, loss = 3.669, time = 0.024
60/125000, loss = 2.902, time = 0.017
70/125000, loss = 5.352, time = 0.017
80/125000, loss = 3.999, time = 0.018
90/125000, loss = 3.985, time = 0.017
SAVED MODEL
100/125000, loss = 5.684, time = 0.025
110/125000, loss = 7.173, time = 0.017
120/125000, loss = 4.206, time = 0.017
130/125000, loss = 5.167, time = 0.017
140/125000, loss = 3.173, time = 0.017
SAVED MODEL
150/125000, loss = 4.503, time = 0.026
160/125000, loss = 4.167, time = 0.019
170/125000, loss = 4.523, time = 0.017
180/125000, loss = 

KeyboardInterrupt: 